## Environment setup

For optimal training speed, ensure to enable the GPU of Colab!
Go to `Runtime>Change runtime type`, then select `GPU` in `Hardware accelerator` and click on `Save`.

In [1]:
!nvidia-smi

Mon Oct 27 14:47:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   54C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      2
  On-line CPU(s) list:       0,1
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:              6
    Model:                   85
    Thread(s) per core:      2
    Core(s) per socket:      1
    Socket(s):               1
    Stepping:                3
    BogoMIPS:                4000.44
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ss h
                             t syscall nx pdpe1gb rdtscp lm constant_tsc rep_goo
                             d nopl xtopology nonstop_tsc cpuid tsc_known_freq p
                             ni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2ap
                   

## Installation

In this tutorial, you will need the entire project codebase. So first, we clone the project's GitHub repository and install from source.


In [3]:
!git clone https://github.com/frgfm/torch-cam.git
!uv pip install -e 'torch-cam/.[scripts]'

Cloning into 'torch-cam'...
remote: Enumerating objects: 7849, done.
remote: Counting objects: 100% (673/673), done.
remote: Compressing objects: 100% (347/347), done.
remote: Total 7849 (delta 531), reused 352 (delta 325), pack-reused 7176 (from 2)
Receiving objects: 100% (7849/7849), 10.79 MiB | 14.09 MiB/s, done.
Resolving deltas: 100% (5573/5573), done.
Using Python 3.12.12 environment at: /usr
Resolved 37 packages in 279ms
Prepared 2 packages in 329ms
Uninstalled 1 package in 44ms
Installed 2 packages in 15ms
 - numpy==2.0.2
 + numpy==1.26.4
 + torchcam==0.5.0.dev0 (from file:///content/torch-cam)


In [4]:
!cd torch-cam && git pull && git checkout frgfm/benchmark-docs && uv pip install -e '.[scripts]'

Already up to date.
Branch 'frgfm/benchmark-docs' set up to track remote branch 'frgfm/benchmark-docs' from 'origin'.
Switched to a new branch 'frgfm/benchmark-docs'
Using Python 3.12.12 environment at: /usr
Resolved 43 packages in 76ms
Prepared 1 package in 5ms
Uninstalled 1 package in 0.31ms
Installed 1 package in 0.93ms
 - torchcam==0.5.0.dev0 (from file:///content/torch-cam)
 + torchcam==0.4.1.dev0 (from file:///content/torch-cam)


Now we'll download the files we need for the script to work

In [5]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-320.tgz
!tar -xvzf imagenette2-320.tgz
!rm imagenette2-320.tgz

Streaming output truncated to the last 5000 lines.
imagenette2-320/train/n03888257/n03888257_16077.JPEG
imagenette2-320/train/n03888257/n03888257_23339.JPEG
imagenette2-320/train/n03888257/n03888257_44204.JPEG
imagenette2-320/train/n03888257/n03888257_61633.JPEG
imagenette2-320/train/n03888257/n03888257_15067.JPEG
imagenette2-320/train/n03888257/n03888257_75365.JPEG
imagenette2-320/train/n03888257/n03888257_63966.JPEG
imagenette2-320/train/n03888257/n03888257_3927.JPEG
imagenette2-320/train/n03888257/n03888257_20684.JPEG
imagenette2-320/train/n03888257/ILSVRC2012_val_00047778.JPEG
imagenette2-320/train/n03888257/n03888257_14016.JPEG
imagenette2-320/train/n03888257/n03888257_37776.JPEG
imagenette2-320/train/n03888257/ILSVRC2012_val_00041706.JPEG
imagenette2-320/train/n03888257/n03888257_17513.JPEG
imagenette2-320/train/n03888257/n03888257_17143.JPEG
imagenette2-320/train/n03888257/n03888257_6738.JPEG
imagenette2-320/train/n03888257/n03888257_4355.JPEG
imagenette2-320/train/n03888257/ILS

## Running the benchmark

The benchmark script has many parameters for you to experiment on, you can use the help command to inspect them!

In [6]:
!python torch-cam/scripts/eval_perf.py --help

usage: eval_perf.py [-h] [--arch ARCH] [--target TARGET] [--size SIZE]
                    [-b BATCH_SIZE] [--device DEVICE] [-j WORKERS]
                    data_path method

CAM method performance evaluation

positional arguments:
  data_path             path to dataset folder
  method                CAM method to use

options:
  -h, --help            show this help message and exit
  --arch ARCH           Name of the torchvision architecture (default:
                        mobilenet_v3_large)
  --target TARGET       Target layer name (default: None)
  --size SIZE           The image input size (default: 224)
  -b BATCH_SIZE, --batch-size BATCH_SIZE
                        batch size (default: 32)
  --device DEVICE       Default device to perform computation on (default:
                        None)
  -j WORKERS, --workers WORKERS
                        number of data loading workers (default: 2)


Now let's run the benchmark on the downloaded dataset for a given model architecture

In [7]:
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ GradCAM --arch mobilenet_v3_large --target features
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ GradCAMpp --arch mobilenet_v3_large --target features
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ SmoothGradCAMpp --arch mobilenet_v3_large --target features
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ XGradCAM --arch mobilenet_v3_large --target features
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ LayerCAM --arch mobilenet_v3_large --target features

# Other CAM methods run out of VRAM

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth
100% 21.1M/21.1M [00:00<00:00, 163MB/s]
GradCAM w/ mobilenet_v3_large (validation set of Imagenette on (224, 224) inputs)
Average Drop 26.78%, Increase in Confidence 34.83%, Skipped 0 samples
GradCAMpp w/ mobilenet_v3_large (validation set of Imagenette on (224, 224) inputs)
Average Drop 31.82%, Increase in Confidence 25.35%, Skipped 0 samples
SmoothGradCAMpp w/ mobilenet_v3_large (validation set of Imagenette on (224, 224) inputs)
Average Drop 26.81%, Increase in Confidence 26.78%, Skipped 0 samples
XGradCAM w/ mobilenet_v3_large (validation set of Imagenette on (224, 224) inputs)
Average Drop 26.78%, Increase in Confidence 34.83%, Skipped 0 samples
LayerCAM w/ mobilenet_v3_large (validation set of Imagenette on (224, 224) inputs)
Average Drop 25.26%, Increase in Confidence 28.82%, Skipped 0 samples


In [8]:
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ GradCAM --arch resnet18 --target layer4
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ GradCAMpp --arch resnet18 --target layer4
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ SmoothGradCAMpp --arch resnet18 --target layer4
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ XGradCAM --arch resnet18 --target layer4
!python torch-cam/scripts/eval_perf.py /content/imagenette2-320/ LayerCAM --arch resnet18 --target layer4

# Other CAM methods run out of VRAM

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 134MB/s]
GradCAM w/ resnet18 (validation set of Imagenette on (224, 224) inputs)
Average Drop 26.86%, Increase in Confidence 22.50%, Skipped 0 samples
GradCAMpp w/ resnet18 (validation set of Imagenette on (224, 224) inputs)
Average Drop 52.71%, Increase in Confidence 19.62%, Skipped 0 samples
SmoothGradCAMpp w/ resnet18 (validation set of Imagenette on (224, 224) inputs)
Average Drop 20.88%, Increase in Confidence 24.99%, Skipped 0 samples
XGradCAM w/ resnet18 (validation set of Imagenette on (224, 224) inputs)
Average Drop 26.86%, Increase in Confidence 22.50%, Skipped 0 samples
LayerCAM w/ resnet18 (validation set of Imagenette on (224, 224) inputs)
Average Drop 18.05%, Increase in Confidence 28.94%, Skipped 0 samples
